In [ ]:
import awkward as ak
import vector
import glob
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
qq_path = glob.glob("/home/hardiveski/ml-tau-en-reg/training-outputs/240626_train_on_z/v1/feats_kin_feats_lifetimes/binary_classification/SimpleDNN/qq_test.parquet")
z_path = glob.glob("/home/hardiveski/ml-tau-en-reg/training-outputs/240626_train_on_z/v1/feats_kin_feats_lifetimes/binary_classification/SimpleDNN/z_test.parquet")
zh_path = glob.glob("/home/hardiveski/ml-tau-en-reg/training-outputs/240626_train_on_z/v1/feats_kin_feats_lifetimes/binary_classification/SimpleDNN/zh_test.parquet")

In [ ]:
qq_data = ak.from_parquet(qq_path)
z_data = ak.from_parquet(z_path)
zh_data = ak.from_parquet(zh_path)

In [ ]:
# Separated prediction and target data
data_pred = {
    'qq': qq_data['binary_classification']['pred'],
    'z': z_data['binary_classification']['pred'],
    'zh': zh_data['binary_classification']['pred']
}

data_target = {
    'qq': qq_data['binary_classification']['target'],
    'z': z_data['binary_classification']['target'],
    'zh': zh_data['binary_classification']['target']
}

In [ ]:
print("Lengths of data_pred:")
for key, value in data_pred.items():
    print(f"{key}: {len(value)}")

print("Lengths of data_target:")
for key, value in data_target.items():
    print(f"{key}: {len(value)}")

In [ ]:
for i, key in enumerate(data_pred.keys()):
    plt.figure(figsize=(4, 3))
    plt.hist(data_pred[key], bins=40, alpha=0.75, color='#234112', edgecolor='black');
    plt.legend([key])
    plt.yscale('log')  
plt.tight_layout()
plt.show()

In [ ]:
# Signal with backround. But what is the signal? Z or ZH or both
signal = {
    'z': ak.concatenate([z_data['binary_classification'], qq_data['binary_classification']]),
    'zh': ak.concatenate([zh_data['binary_classification'], qq_data['binary_classification']]),
    'both': ak.concatenate([z_data['binary_classification'],zh_data['binary_classification'], qq_data['binary_classification']])
}

In [ ]:
# ROC curve
keys =[]
plt.figure(figsize=(5,4.5))
for i, key in enumerate(signal):
    fpr, tpr, _ = roc_curve(signal[key]['target'], signal[key]['pred'])
    plt.plot(tpr, fpr)
    plt.yscale('log')
    keys.append(key)
    # Uncomment to see all the plots in different figure
    #plt.show()

plt.xlabel('TPR')
plt.ylabel('FPR')
plt.tight_layout()

plt.legend(keys)
plt.show()